In [1]:
import pandas as pd
import glob
import random
import numpy as np
import pickle

import matplotlib.pyplot as plt
import ast
from wordcloud import WordCloud
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import gensim
from gensim import corpora

In [2]:
np.random.seed = 42
random.seed = 42

## 후보/연도별로 연설 합치기

In [3]:
df = None
for file in glob.glob('./data/test_preprocess/*'):
    df_curr = pd.read_csv(file, index_col=0)
    df_curr["Year"] = int(file.split("\\")[-1][0:4])
    if df is None:
        df = df_curr
    else:
        df = pd.concat([df,df_curr])


#ValueError: invalid literal for int() with base 10: 'Nomi' 나와도 정상(맨마지막 파일이라 실질 영향 X고 미포함한게 맞음)

ValueError: invalid literal for int() with base 10: 'Nomi'

In [ ]:
df

## 연도 분포 확인

In [ ]:


fig, ax = plt.subplots()

df.groupby("Year")["Year"].value_counts().plot.bar(ax=ax)
# ax.hist(df["Year"], bins=range(1920,2024,4))

## 후보별 분포 확인

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

df.groupby("name")["name"].value_counts().plot.bar(ax=ax)
# ax.hist(df["name"])
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

plt.show()

In [ ]:
df["joined_speech"] = df["speech"].apply(lambda l: ' '.join(ast.literal_eval(l)))

## 길이 분포 확인

In [ ]:
fig, ax = plt.subplots()

ax.hist(df["joined_speech"].apply(lambda l: len(l.split(' '))))

In [ ]:
df.info()

In [ ]:

ast.literal_eval(df["speech"].values[0])

## WordCloud

In [ ]:
# !conda install wordcloud -y


df_test = pd.read_csv('./data/orig/2020_Trump_speech.csv', index_col=0)

speeches = list(map(lambda l: ' '.join(ast.literal_eval(l)), df_test["speech"].values))



speeches_processed = list()
for word in speeches:
    if word not in stop_words_list:
        speeches_processed.append(word)

speeches_combined = ' '.join(speeches_processed)

In [ ]:
wordcloud = WordCloud(
    background_color = 'black',
    width = 1000, height = 500).generate(speeches_combined)
plt.figure(figsize = (15, 10))
plt.imshow(wordcloud)
plt.axis('off')
plt.draw()

In [ ]:
df_test = pd.read_csv('./data/to_preprocess/2020_Biden_speech_edited.csv', index_col=0)

In [ ]:
speeches = list(map(lambda l: ' '.join(ast.literal_eval(l)), df_test["speech"].values))

speeches_combined = ' '.join(speeches)

speeches_processed = list()
for word in speeches:
    if word not in stop_words_list:
        speeches_processed.append(word)

speeches_combined = ' '.join(speeches_processed)

In [ ]:
wordcloud = WordCloud(
    background_color = 'black',
    width = 1000, height = 500).generate(speeches_combined)
plt.figure(figsize = (15, 10))
plt.imshow(wordcloud)
plt.axis('off')
plt.draw()

## LDA

In [ ]:

lambda x: re.sub('[,\.!?]','',x)

lambda x: x.lower()



In [ ]:
import gensim
from gensim_utils import simple_preprocess
import nltk


In [5]:
# !conda install nltk -y

# nltk.download('stopwords')
# nltk.download("punkt")
stop_words_list = stopwords.words('english')

In [5]:
len(stop_words_list)

179

In [6]:
df["speech_whole"] = df["speech"].apply(lambda l: ' '.join(ast.literal_eval(l)))
# ast.literal_eval(df["speech"].values[0])

In [7]:
def filter_stopwords(token_list):
    filtered_list = list()
    for token in token_list:
        if token not in stop_words_list:
            filtered_list.append(token)
    return filtered_list

In [8]:
def filter_externals(string):
    # externals_list = ['[applause]', '(applause)']
    # for external in externals_list:
    #     string = string.replace(external, '')
    string = re.sub('\s*\[[^]]*\]', '', string)
    string = re.sub('\s*\([^)]*\)', '', string)
    string = re.sub('\s*\([^]]*\]', '', string)
    string = re.sub('\s*\[[^)]*\)', '', string)
    return string

In [9]:
def filter_nonwords(string):
    out = string
    out = re.sub('[-–—–;\.!?]', ' ', out)
    out = re.sub('[,"\'`]','', out)
    out = re.sub('[^a-zA-Z0-9\s\-_./]','', out)
    out = out.lower()
    return out

In [10]:
# 청중 호응 부분 제거

df["speech_processed"] = df["speech_whole"].apply(filter_externals)

In [11]:
# for idx, speech in enumerate(df["speech_processed"].values):
#     found_idx = speech.find("applause")
#     if found_idx != -1:
#         print(speech[found_idx-50:found_idx+15])

ours for which they deserve on every occasion the applause of pat
to say that the appeal of this Administration for applause for it
e of the briefness of the time, let's cut out the applause. QUEST
do. The decision is yours. Boos, shouts, screams, applause on Nov
Nixon is your man. Response from the audience and applause.] But 
is up. I want to thank you for your very generous applause, for y
e if we go to work in the State of Missouri. Your applause drowne
d particularly I think we ought to express by our applause our ap
er, and I'm glad to see you know him well by your applause, and a
- and we're going to keep it that way. Cheers and applause.] I am
f these men are patriots and they all deserve the applause of a c
 are now beginning our fifth year of anticipatory applause. Nowad
 phase of school integration. I'm not looking for applause and do
u always make a criticism and get a great deal of applause for it
it more than once?" I said, "Well, that's my best applause line, 
lice chief

In [12]:
#문법요소 제거

df["speech_processed"] = df["speech_processed"].apply(filter_nonwords)

In [13]:
#다른 토큰화 방식 사용할 경우
# from nltk.tokenize.treebank import TreebankWordTokenizer
# t = TreebankWordTokenizer()

# df["speech_processed"] = df["speech_processed"].apply(t.tokenize)

df["speech_processed"] = df["speech_processed"].apply(word_tokenize)

In [14]:
#기본 불용어 제거

df["speech_processed"] = df["speech_processed"].apply(filter_stopwords)

In [14]:
# !conda install gensim -y

dictionary = corpora.Dictionary(df["speech_processed"])

corpus = [dictionary.doc2bow(text) for text in df["speech_processed"]]

In [15]:
# 잘못된 단어 있나 확인
# words = list(dictionary.values())
# words.sort(key=lambda x:len(x), reverse=True)
# words

['com/page/community/post/stateupdates/gg5lvb',
 'veterinarian/taxidermist',
 'com/issues/healthcare/fc',
 'superdelegates/endorsers',
 'aid/20081102/opinions0',
 'housekeeper/babysitter',
 'uncharacteristically',
 'counterproliferation',
 'charlestoncharleston',
 'mischaracterizations',
 'bluecross/blueshield',
 'overhospitalization',
 'straightforwardness',
 'republicancandidate',
 'intergenerationally',
 'israeli/palestinian',
 'impulsivedisdainful',
 'bidenadministration',
 'misrepresentations',
 'characteristically',
 'overcapitalization',
 'chairmanrepublican',
 'transcendentalists',
 'antidiscrimination',
 'disproportionately',
 'irresponsibilities',
 'overcentralization',
 'interrelationships',
 'oversimplification',
 'televisionstations',
 'ultraconservatives',
 'disenfranchisement',
 'telecommunications',
 'entrepreneurialism',
 'institutionalizing',
 'timescharacterized',
 'interconnectedness',
 'unconstitutionally',
 'counterprogramming',
 'instrumentalities',
 'constitutio

In [ ]:
n_topics = 15
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=n_topics, id2word=dictionary, passes=15, random_state=42)
topics = lda_model.print_topics(num_words=5)

for topic in topics:
    print(topic)

In [16]:
n_topics = 20
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=n_topics, id2word=dictionary, passes=15, random_state=42)
topics = lda_model.print_topics(num_words=5)

for topic in topics:
    print(topic)

(0, '0.021*"years" + 0.019*"4" + 0.012*"president" + 0.011*"well" + 0.010*"audience"')
(1, '0.004*"strauss" + 0.004*"vacation" + 0.004*"surge" + 0.003*"drilling" + 0.003*"jepsen"')
(2, '0.010*"jobs" + 0.010*"america" + 0.008*"thats" + 0.008*"tax" + 0.008*"american"')
(3, '0.016*"would" + 0.015*"government" + 0.012*"governor" + 0.011*"federal" + 0.010*"carter"')
(4, '0.017*"people" + 0.015*"republican" + 0.014*"congress" + 0.014*"democratic" + 0.010*"party"')
(5, '0.016*"people" + 0.012*"years" + 0.011*"want" + 0.009*"thank" + 0.008*"work"')
(6, '0.025*"dole" + 0.015*"tax" + 0.014*"going" + 0.014*"america" + 0.012*"clinton"')
(7, '0.010*"us" + 0.009*"america" + 0.008*"must" + 0.007*"people" + 0.006*"world"')
(8, '0.013*"war" + 0.013*"iraq" + 0.008*"world" + 0.008*"security" + 0.008*"president"')
(9, '0.018*"president" + 0.009*"senator" + 0.009*"members" + 0.009*"health" + 0.008*"security"')
(10, '0.018*"country" + 0.016*"united" + 0.016*"states" + 0.013*"think" + 0.012*"mr"')
(11, '0.01

In [ ]:
topics = lda_model.print_topics(num_words=10)

for topic in topics:
    print(topic)

In [ ]:
for i, topic_list in enumerate(lda_model[corpus]):
    print(i,'번째 문서의 topic 비율은',topic_list)

## Calculate TF-IDF

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer


v = TfidfVectorizer(use_idf=True)


x = v.fit_transform(df["speech_processed"].apply(lambda x: ' '.join(x)))

In [27]:
v.vocabulary_

{'mr': 24769,
 'chairman': 7078,
 'delegates': 10508,
 'desired': 10900,
 'nomination': 25536,
 'wished': 40795,
 'remain': 30976,
 'bench': 4672,
 'critical': 9619,
 'period': 27637,
 'national': 25096,
 'history': 18029,
 'recognize': 30394,
 'right': 31788,
 'summon': 36045,
 'paramount': 27118,
 'duty': 12449,
 'respond': 31401,
 'speak': 34745,
 'time': 37329,
 'exigency': 13926,
 'transcending': 37775,
 'merely': 23803,
 'partisan': 27214,
 'consideration': 8770,
 'voice': 39853,
 'demand': 10584,
 'dominant': 11968,
 'thorough': 37134,
 'going': 16456,
 'americanism': 2717,
 'firm': 14875,
 'protective': 29386,
 'upbuilding': 39201,
 'policies': 28353,
 'essential': 13580,
 'peace': 27418,
 'security': 33086,
 'call': 6406,
 'crisis': 9607,
 'fail': 14225,
 'answer': 2950,
 'pledge': 28209,
 'service': 33325,
 'country': 9348,
 'therefore': 37054,
 'accept': 1774,
 'stand': 35145,
 'unflinching': 38752,
 'maintenance': 22852,
 'rights': 31798,
 'american': 2715,
 'citizens': 758

In [38]:
x.mean(axis=0)
# x.shape

(1, 41373)

In [48]:
from tqdm import tqdm
tfidf_list = list()
for key, val in tqdm(v.vocabulary_.items()):
    tfidf_list.append((key, x.mean(axis=0)[0,val]))

100%|██████████| 41373/41373 [08:50<00:00, 81.79it/s]


In [47]:
len(tfidf_list)

13170

In [49]:
tfidf_list.sort(key=lambda x: x[1], reverse=True)

In [57]:
tfidf_list[0:10]

[('people', 0.06675414747414131),
 ('america', 0.051002382374821235),
 ('going', 0.04853274989066755),
 ('country', 0.0469439220353535),
 ('president', 0.04466549957914515),
 ('want', 0.043198532241879514),
 ('one', 0.04196880085540473),
 ('years', 0.0419114086266054),
 ('know', 0.04141840901691906),
 ('us', 0.03874385278941233)]

In [58]:
tfidf_list[-10:]

[('miklaszewski', 2.5732491441016285e-06),
 ('couches', 2.5732491441016285e-06),
 ('incredulous', 2.5732491441016285e-06),
 ('faiyad', 2.5732491441016285e-06),
 ('publicitywise', 2.5732491441016285e-06),
 ('mcconnells', 2.5732491441016285e-06),
 ('lat', 2.5732491441016285e-06),
 ('pedals', 2.5732491441016285e-06),
 ('thatve', 2.5732491441016285e-06),
 ('berman', 2.5732491441016285e-06)]

## Calculate IDF

In [16]:
idf = v.idf_

from tqdm import tqdm
idf_list = list()
for key, val in tqdm(v.vocabulary_.items()):
    idf_list.append((key, idf[val]))

idf_list.sort(key=lambda x: x[1], reverse=True)

100%|██████████| 41373/41373 [00:00<00:00, 1451910.90it/s]


In [64]:
idf_list[-50:]

[('take', 1.411317026676103),
 ('year', 1.4068125145549983),
 ('americans', 1.4041195147228065),
 ('better', 1.4018808748803493),
 ('dont', 1.4005400925565843),
 ('also', 1.3996472353544118),
 ('need', 1.3960837571960234),
 ('right', 1.3912046168890075),
 ('back', 1.381517178733318),
 ('state', 1.3767084262974345),
 ('thank', 1.3710550083263349),
 ('help', 1.3680240469238998),
 ('said', 1.3671597426350046),
 ('go', 1.3650022444949834),
 ('get', 1.361130439270998),
 ('government', 1.357701374616481),
 ('much', 1.3419931339857256),
 ('think', 1.3411510286728034),
 ('work', 1.3394689424898185),
 ('like', 1.3331861993103233),
 ('believe', 1.3298514734188702),
 ('last', 1.322801890004607),
 ('many', 1.3162120803255513),
 ('come', 1.3158016555301328),
 ('good', 1.3153913991141222),
 ('well', 1.3121153929082214),
 ('say', 1.309665412099237),
 ('way', 1.309665412099237),
 ('going', 1.29228070096972),
 ('first', 1.2918799796660208),
 ('world', 1.2902786982990468),
 ('united', 1.2759813935982225

In [20]:
# import pickle
# with open("./data/idflist","wb") as fp:
#     pickle.dump(idf_list, fp)

## 2차 정제본

In [31]:
with open("./data/idflist","rb") as fp:
    idf_list = pickle.load(fp)[-50:]

# len(filter_words)

In [32]:
filter_words = list()
for idf in idf_list[-50:]:
    filter_words.append(idf[0])

In [33]:
def filter_lowidf(token_list):
    filtered_list = list()

    for token in token_list:
        if token not in filter_words:
            filtered_list.append(token)
    return filtered_list

In [34]:
df["speech_processed2"] = df["speech_processed"].apply(filter_lowidf)

100%|██████████| 3011/3011 [00:00<00:00, 1204946.98it/s]


In [98]:
df.to_csv("./data/speech_combined.csv")

## 2차 정제본으로 LDA

In [35]:
# !conda install gensim -y

dictionary = corpora.Dictionary(df["speech_processed2"])

corpus = [dictionary.doc2bow(text) for text in df["speech_processed2"]]

In [36]:
n_topics = 20
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=n_topics, id2word=dictionary, passes=15, random_state=42)
topics = lda_model.print_topics(num_words=5)

for topic in topics:
    print(topic)

(0, '0.011*"peace" + 0.008*"must" + 0.006*"war" + 0.006*"friends" + 0.006*"mr"')
(1, '0.056*"care" + 0.053*"health" + 0.017*"insurance" + 0.013*"medical" + 0.012*"costs"')
(2, '0.022*"energy" + 0.016*"oil" + 0.006*"future" + 0.006*"economy" + 0.006*"power"')
(3, '0.012*"thats" + 0.012*"jobs" + 0.011*"tax" + 0.009*"economy" + 0.008*"senator"')
(4, '0.013*"war" + 0.010*"iraq" + 0.010*"must" + 0.006*"security" + 0.006*"military"')
(5, '0.009*"members" + 0.009*"health" + 0.008*"audience" + 0.008*"security" + 0.007*"senator"')
(6, '0.012*"federal" + 0.008*"economic" + 0.006*"percent" + 0.006*"tax" + 0.006*"business"')
(7, '0.011*"got" + 0.010*"im" + 0.009*"weve" + 0.007*"sure" + 0.005*"see"')
(8, '0.025*"atomic" + 0.002*"defendants" + 0.002*"bomb" + 0.001*"viejo" + 0.001*"bounced"')
(9, '0.012*"clinton" + 0.011*"im" + 0.009*"thats" + 0.009*"got" + 0.006*"lot"')
(10, '0.047*"farm" + 0.035*"farmers" + 0.020*"farmer" + 0.018*"program" + 0.017*"agriculture"')
(11, '0.156*"mccain" + 0.041*"sen" 

In [100]:
n_topics = 20
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=n_topics, id2word=dictionary, passes=15, random_state=42)
topics = lda_model.print_topics(num_words=5)

for topic in topics:
    print(topic)

(0, '0.011*"peace" + 0.008*"must" + 0.006*"war" + 0.006*"friends" + 0.006*"mr"')
(1, '0.056*"care" + 0.053*"health" + 0.017*"insurance" + 0.013*"medical" + 0.012*"costs"')
(2, '0.022*"energy" + 0.016*"oil" + 0.006*"future" + 0.006*"economy" + 0.006*"power"')
(3, '0.012*"thats" + 0.012*"jobs" + 0.011*"tax" + 0.009*"economy" + 0.008*"senator"')
(4, '0.013*"war" + 0.010*"iraq" + 0.010*"must" + 0.006*"security" + 0.006*"military"')
(5, '0.009*"members" + 0.009*"health" + 0.008*"audience" + 0.008*"security" + 0.007*"senator"')
(6, '0.012*"federal" + 0.008*"economic" + 0.006*"percent" + 0.006*"tax" + 0.006*"business"')
(7, '0.011*"got" + 0.010*"im" + 0.009*"weve" + 0.007*"sure" + 0.005*"see"')
(8, '0.025*"atomic" + 0.002*"defendants" + 0.002*"bomb" + 0.001*"viejo" + 0.001*"bounced"')
(9, '0.012*"clinton" + 0.011*"im" + 0.009*"thats" + 0.009*"got" + 0.006*"lot"')
(10, '0.047*"farm" + 0.035*"farmers" + 0.020*"farmer" + 0.018*"program" + 0.017*"agriculture"')
(11, '0.156*"mccain" + 0.041*"sen" 

In [29]:
# words = list(dictionary.values())
# words.sort(key=lambda x:len(x), reverse=True)
# words